In [1]:
# Se importan librerias a utilizar
import os
import requests
import json
from datetime import datetime, timedelta
import pandas as pd
import pandas_profiling
from pandas_profiling import ProfileReport
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MultiLabelBinarizer
import sqlite3
from sqlalchemy import create_engine



/var/folders/9b/spdqtg_n32b9hgq_n7ttcb3w0000gn/T/ipykernel_23671/2237484517.py:7: DeprecationWarning: `import pandas_profiling` is going to be deprecated by April 1st. Please use `import ydata_profiling` instead.
  import pandas_profiling


In [2]:
# Definir la ruta de los directorios
json_dir = '../json'
profiling_dir = '../profiling'
data_dir = '../data'
db_dir = '../db'
model_dir ='../model'

In [3]:
# Fecha de inicio y fin
start_date = datetime(2022, 12, 1)
end_date = datetime(2023, 1, 1)

current_date = start_date

# Lista para almacenar todos los datos de las series
all_shows = []

In [4]:
while current_date < end_date:
    # Formatear la fecha en el formato correcto (yyyy-mm-dd)
    date_str = current_date.strftime("%Y-%m-%d")

    # Construir la URL para la petición
    url = f"http://api.tvmaze.com/schedule/web?date={date_str}"

    # Enviar la petición GET
    response = requests.get(url)

    # Verificar que la petición fue exitosa
    if response.status_code == 200:
        # Convertir la respuesta a JSON
        shows = response.json()

        # Agregar los shows del día a la lista total de shows
        all_shows.extend(shows)
    else:
        print(f"Error obteniendo datos para la fecha: {date_str}")

    # Avanzar al siguiente día
    current_date += timedelta(days=1)

# Almacenar los datos crudos en un archivo JSON
# Crear la subcarpeta si no existe
if not os.path.exists(json_dir):
    os.makedirs(json_dir)
    
with open(f"{json_dir}/all_shows_december_2022.json", "w") as file:
    json.dump(all_shows, file)

In [5]:
# Leer los datos JSON del archivo
with open(f"{json_dir}/all_shows_december_2022.json", "r") as file:
    data = json.load(file)

# Para la comodidad en el manejo, se convierte todo en un unico dataframe inicialmente (sep _ para
# crear las columnas en el dataframe de las llaves anidadas)
data_df = pd.json_normalize(data, sep="_")

# Ahora, dividimos el dataframe general en 7 dataframes diferentes basados en los campos del JSON

# DataFrame de episodios
episodes_df = data_df[[
    'id',
    'url',
    'name',
    'season',
    'number',
    'type',
    'airdate',
    'airtime',
    'airstamp',
    'runtime',
    '_links_self_href',
    '_links_show_href'
]]

# DataFrame de calificaciones de episodios
episode_ratings_df = data_df[[
    'id',
    'rating_average'
]]

# DataFrame de shows
shows_df = data_df[[
    '_embedded_show_id',
    '_embedded_show_url',
    '_embedded_show_name',
    '_embedded_show_type',
    '_embedded_show_language',
    '_embedded_show_genres',
    '_embedded_show_status',
    '_embedded_show_runtime',
    '_embedded_show_averageRuntime',
    '_embedded_show_premiered',
    '_embedded_show_officialSite',
    '_embedded_show_weight',
    '_embedded_show_updated',
    '_embedded_show__links_self_href',
    '_embedded_show__links_previousepisode_href'
]]

# DataFrame de horario del show
show_schedule_df = data_df[[
    '_embedded_show_id',
    '_embedded_show_schedule_time',
    '_embedded_show_schedule_days'
]]

# DataFrame de calificación del show
show_ratings_df = data_df[[
    '_embedded_show_id',
    '_embedded_show_rating_average'
]]

# DataFrame de webChannel
webchannel_df = data_df[[
    '_embedded_show_id',
    '_embedded_show_webChannel_id',
    '_embedded_show_webChannel_name',
    '_embedded_show_webChannel_country_name',
    '_embedded_show_webChannel_country_code',
    '_embedded_show_webChannel_country_timezone',
    '_embedded_show_webChannel_officialSite'
]]

In [6]:
# Se crea un profiling inicial de cada uno de los dataframes, apoyados de la libreria
# pandas_profiling, el cual genera por si mismo archivos html con un resumen general de
# cada uno de los d ataframes. 

# Crea la subcarpeta si no existe
if not os.path.exists(profiling_dir):
    os.makedirs(profiling_dir)

# Crear perfiles para cada dataframe
df_list = [episodes_df, episode_ratings_df, shows_df, show_schedule_df, show_ratings_df, webchannel_df]
df_names = ['episodes', 'episode_ratings', 'shows', 'show_schedule', 'show_ratings', 'webchannel']

for df, name in zip(df_list, df_names):
    profile = pandas_profiling.ProfileReport(df)
    profile.to_file(f"{profiling_dir}/{name}_profile.html")

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/ydata_profiling/utils/dataframe.py:137: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={"index": "df_index"}, inplace=True)


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

...

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/ydata_profiling/utils/dataframe.py:137: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={"index": "df_index"}, inplace=True)


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [7]:
# Seleccionamos el dataframe con el que vamos a trabajar
df = episodes_df  

#Informacion general de variables

df.info()

# Obtener la dimensión del dataframe
print("Shape:", df.shape)

# Obtener la lista de columnas
print("Columns:", df.columns.tolist())

# Obtener la cantidad de valores nulos por columna
print("Missing values:", df.isna().sum())

# Obtener la cantidad de valores únicos por columna
print("Unique values:", df.nunique())

# Obtener la descripción estadística de las columnas numéricas
print("Describe:\n", df.describe())

# Obtener el conteo de valores para columnas categóricas
for column in df.select_dtypes(include=['object']).columns:
    print(f"Value counts for {column}:\n", df[column].value_counts())

# Verificar si hay filas duplicadas
print("Duplicate rows:", df.duplicated().sum())
if df.duplicated().sum() > 0:
    print("Here are the duplicated rows:")
    print(df[df.duplicated(keep=False)])  # Mostrar las filas duplicadas

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3991 entries, 0 to 3990
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                3991 non-null   int64  
 1   url               3991 non-null   object 
 2   name              3991 non-null   object 
 3   season            3991 non-null   int64  
 4   number            3907 non-null   float64
 5   type              3991 non-null   object 
 6   airdate           3991 non-null   object 
 7   airtime           3991 non-null   object 
 8   airstamp          3991 non-null   object 
 9   runtime           3735 non-null   float64
 10  _links_self_href  3991 non-null   object 
 11  _links_show_href  3991 non-null   object 
dtypes: float64(2), int64(2), object(8)
memory usage: 374.3+ KB
Shape: (3991, 12)
Columns: ['id', 'url', 'name', 'season', 'number', 'type', 'airdate', 'airtime', 'airstamp', 'runtime', '_links_self_href', '_links_show_href']
Missing v

In [8]:
# Seleccionamos el dataframe con el que vamos a trabajar
df = shows_df 

#Informacion general de variables

df.info()

# Obtener la dimensión del dataframe
print("Shape:", df.shape)

# Obtener la lista de columnas
print("Columns:", df.columns.tolist())

# Obtener la cantidad de valores nulos por columna
print("Missing values:", df.isna().sum())

# Obtener la cantidad de valores únicos por columna
#print("Unique values:", df.nunique())

# Obtener la descripción estadística de las columnas numéricas
print("Describe:\n", df.describe())

# Obtener el conteo de valores para columnas categóricas
for column in df.select_dtypes(include=['object']).columns:
    print(f"Value counts for {column}:\n", df[column].value_counts())



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3991 entries, 0 to 3990
Data columns (total 15 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   _embedded_show_id                           3991 non-null   int64  
 1   _embedded_show_url                          3991 non-null   object 
 2   _embedded_show_name                         3991 non-null   object 
 3   _embedded_show_type                         3991 non-null   object 
 4   _embedded_show_language                     3889 non-null   object 
 5   _embedded_show_genres                       3991 non-null   object 
 6   _embedded_show_status                       3991 non-null   object 
 7   _embedded_show_runtime                      1890 non-null   float64
 8   _embedded_show_averageRuntime               3754 non-null   float64
 9   _embedded_show_premiered                    3991 non-null   object 
 10  _embedded_sh

In [9]:
# Seleccionamos el dataframe con el que vamos a trabajar
df = show_schedule_df 

#Informacion general de variables

df.info()

# Obtener la dimensión del dataframe
print("Shape:", df.shape)

# Obtener la lista de columnas
print("Columns:", df.columns.tolist())

# Obtener la cantidad de valores nulos por columna
print("Missing values:", df.isna().sum())

# Obtener la cantidad de valores únicos por columna
#print("Unique values:", df.nunique())

# Obtener la descripción estadística de las columnas numéricas
print("Describe:\n", df.describe())

# Obtener el conteo de valores para columnas categóricas
for column in df.select_dtypes(include=['object']).columns:
    print(f"Value counts for {column}:\n", df[column].value_counts())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3991 entries, 0 to 3990
Data columns (total 3 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   _embedded_show_id             3991 non-null   int64 
 1   _embedded_show_schedule_time  3991 non-null   object
 2   _embedded_show_schedule_days  3991 non-null   object
dtypes: int64(1), object(2)
memory usage: 93.7+ KB
Shape: (3991, 3)
Columns: ['_embedded_show_id', '_embedded_show_schedule_time', '_embedded_show_schedule_days']
Missing values: _embedded_show_id               0
_embedded_show_schedule_time    0
_embedded_show_schedule_days    0
dtype: int64
Describe:
        _embedded_show_id
count        3991.000000
mean        57275.924330
std         14870.602428
min            81.000000
25%         54476.000000
50%         64208.000000
75%         65995.000000
max         70410.000000
Value counts for _embedded_show_schedule_time:
 _embedded_show_schedule_ti

In [10]:
# MODIFICACION DE DATOS episodes_df

# Pasar a indice 
episodes_df.set_index('id', inplace=True)


# Eliminar las filas analizadas
episodes_df = episodes_df[(episodes_df['season'] != 2022) & (episodes_df['season'] != 2021)]
episodes_df = episodes_df.dropna(subset=['number'])
episodes_df = episodes_df.dropna(subset=['runtime'])
episodes_df = episodes_df[(episodes_df['runtime'] > 1) & (episodes_df['runtime'] <= 180)]


# Cambiar el tipo de datos 
episodes_df['type'] = episodes_df['type'].astype('category')
episodes_df['airstamp'] = pd.to_datetime(episodes_df['airstamp'], errors='coerce')

# Eliminar columnas
episodes_df = episodes_df.drop(columns=['airdate','airtime'],)
 



In [11]:
# MODIFICACION DE DATOS episode_ratings

# Pasar a indice 
episode_ratings_df.set_index('id', inplace=True)

In [12]:
# MODIFICACION DE DATOS shows_df

# Eliminar los valores duplicados utilizando la columna id como argumento
# intentando eliminar aquellos con menos valores nulos. 

# Calcula la cantidad de valores nulos por fila
shows_df['null_count'] = shows_df.isnull().sum(axis=1)
# Ordena por la cantidad de valores nulos, de manera que las filas con menos valores nulos queden primero
shows_df.sort_values('null_count', inplace=True)
# Elimina los duplicados basados en la columna "_embedded_show_id", manteniendo la primera ocurrencia
shows_df = shows_df.drop_duplicates(subset=['_embedded_show_id'])
# Elimina la columna 'null_count'
shows_df.drop(columns=['null_count'], inplace=True)

# Pasar a indice 
shows_df.set_index('_embedded_show_id', inplace=True)

# Eliminar filas con datos faltantes
shows_df = shows_df.dropna(subset=['_embedded_show_averageRuntime'])
shows_df = shows_df.dropna(subset=['_embedded_show_officialSite'])

# Crear una representación one-hot de los géneros
mlb = MultiLabelBinarizer()
genres_onehot = mlb.fit_transform(shows_df['_embedded_show_genres'])
genres_df = pd.DataFrame(genres_onehot, columns=mlb.classes_, index=shows_df.index)
shows_df = shows_df.join(genres_df)

# Eliminar la columnas
shows_df = shows_df.drop(columns=['_embedded_show_genres','_embedded_show_runtime','_embedded_show_updated','_embedded_show__links_previousepisode_href'])


# Cambiar tipo de datos
shows_df['_embedded_show_status'] = shows_df['_embedded_show_status'].astype('category')

#Renombrar columna
shows_df.rename(columns={"Science-Fiction": "Science_Fiction"}, inplace=True)




/var/folders/9b/spdqtg_n32b9hgq_n7ttcb3w0000gn/T/ipykernel_23671/1301326283.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shows_df['null_count'] = shows_df.isnull().sum(axis=1)
/var/folders/9b/spdqtg_n32b9hgq_n7ttcb3w0000gn/T/ipykernel_23671/1301326283.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shows_df.sort_values('null_count', inplace=True)
/var/folders/9b/spdqtg_n32b9hgq_n7ttcb3w0000gn/T/ipykernel_23671/1301326283.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentati

In [13]:
# MODIFICACION DE DATOS show_schedule_df

# Eliminar los valores duplicados utilizando la columna id como argumento
# intentando eliminar aquellos con menos valores nulos. 

# Calcula la cantidad de valores nulos por fila
show_schedule_df['null_count'] = show_schedule_df.isnull().sum(axis=1)
# Ordena por la cantidad de valores nulos, de manera que las filas con menos valores nulos queden primero
show_schedule_df.sort_values('null_count', inplace=True)
# Elimina los duplicados basados en la columna "_embedded_show_id", manteniendo la primera ocurrencia
show_schedule_df = show_schedule_df.drop_duplicates(subset=['_embedded_show_id'])
# Elimina la columna 'null_count'
show_schedule_df.drop(columns=['null_count'], inplace=True)

# Pasar a indice 
show_schedule_df.set_index('_embedded_show_id', inplace=True)

# Convertir la lista de la variable schedule_days en una cadena de texto, separando los días con una coma
show_schedule_df['_embedded_show_schedule_days'] = show_schedule_df['_embedded_show_schedule_days'].apply(', '.join)


/var/folders/9b/spdqtg_n32b9hgq_n7ttcb3w0000gn/T/ipykernel_23671/2618359704.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  show_schedule_df['null_count'] = show_schedule_df.isnull().sum(axis=1)
/var/folders/9b/spdqtg_n32b9hgq_n7ttcb3w0000gn/T/ipykernel_23671/2618359704.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  show_schedule_df.sort_values('null_count', inplace=True)
/var/folders/9b/spdqtg_n32b9hgq_n7ttcb3w0000gn/T/ipykernel_23671/2618359704.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the ca

In [14]:
# MODIFICACION DE DATOS show_ratings_df

# Eliminar los valores duplicados utilizando la columna id como argumento
# intentando eliminar aquellos con menos valores nulos. 

# Calcula la cantidad de valores nulos por fila
show_ratings_df['null_count'] = show_ratings_df.isnull().sum(axis=1)
# Ordena por la cantidad de valores nulos, de manera que las filas con menos valores nulos queden primero
show_ratings_df.sort_values('null_count', inplace=True)
# Elimina los duplicados basados en la columna "_embedded_show_id", manteniendo la primera ocurrencia
show_ratings_df = show_ratings_df.drop_duplicates(subset=['_embedded_show_id'])
# Elimina la columna 'null_count'
show_ratings_df.drop(columns=['null_count'], inplace=True)

# Pasar a indice 
show_ratings_df.set_index('_embedded_show_id', inplace=True)

/var/folders/9b/spdqtg_n32b9hgq_n7ttcb3w0000gn/T/ipykernel_23671/3313493269.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  show_ratings_df['null_count'] = show_ratings_df.isnull().sum(axis=1)
/var/folders/9b/spdqtg_n32b9hgq_n7ttcb3w0000gn/T/ipykernel_23671/3313493269.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  show_ratings_df.sort_values('null_count', inplace=True)
/var/folders/9b/spdqtg_n32b9hgq_n7ttcb3w0000gn/T/ipykernel_23671/3313493269.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the cavea

In [15]:
# MODIFICACION DE DATOS webchannel_df_df

# Eliminar los valores duplicados utilizando la columna id como argumento
# intentando eliminar aquellos con menos valores nulos. 

# Calcula la cantidad de valores nulos por fila
webchannel_df['null_count'] = webchannel_df.isnull().sum(axis=1)
# Ordena por la cantidad de valores nulos, de manera que las filas con menos valores nulos queden primero
webchannel_df.sort_values('null_count', inplace=True)
# Elimina los duplicados basados en la columna "_embedded_show_id", manteniendo la primera ocurrencia
webchannel_df = webchannel_df.drop_duplicates(subset=['_embedded_show_id'])
# Elimina la columna 'null_count'
webchannel_df.drop(columns=['null_count'], inplace=True)

# Pasar a indice 
webchannel_df.set_index('_embedded_show_id', inplace=True)

# Eliminar filas con datos faltantes
webchannel_df = webchannel_df.dropna(subset=['_embedded_show_webChannel_id'])
webchannel_df = webchannel_df.dropna(subset=['_embedded_show_webChannel_name'])

# Eliminar columnas
webchannel_df = webchannel_df.drop(columns=['_embedded_show_webChannel_country_code', '_embedded_show_webChannel_country_timezone','_embedded_show_webChannel_officialSite'],)

/var/folders/9b/spdqtg_n32b9hgq_n7ttcb3w0000gn/T/ipykernel_23671/2187286307.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  webchannel_df['null_count'] = webchannel_df.isnull().sum(axis=1)
/var/folders/9b/spdqtg_n32b9hgq_n7ttcb3w0000gn/T/ipykernel_23671/2187286307.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  webchannel_df.sort_values('null_count', inplace=True)
/var/folders/9b/spdqtg_n32b9hgq_n7ttcb3w0000gn/T/ipykernel_23671/2187286307.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in 

In [16]:
shows_df.head()

,_embedded_show_url,_embedded_show_name,_embedded_show_type,_embedded_show_language,_embedded_show_status,_embedded_show_averageRuntime,_embedded_show_premiered,_embedded_show_officialSite,_embedded_show_weight,_embedded_show__links_self_href,...,Mystery,Nature,Romance,Science_Fiction,Sports,Supernatural,Thriller,Travel,War,Western
_embedded_show_id,,,,,,,,,,,,,,,,,,,,,
55724,https://www.tvmaze.com/shows/55724/hocu-vse-znat,Хочу все знать!,Documentary,Russian,Running,12.0,2021-06-01,https://hd.kinopoisk.ru/film/4615b1810bb9c3bdb...,5,https://api.tvmaze.com/shows/55724,...,0,0,0,0,0,0,0,0,0,0
33902,https://www.tvmaze.com/shows/33902/pop-team-epic,Pop Team Epic,Animation,Japanese,Running,25.0,2018-01-06,http://hoshiiro.jp,65,https://api.tvmaze.com/shows/33902,...,0,0,0,0,0,0,0,0,0,0
64321,https://www.tvmaze.com/shows/64321/bocchi-the-...,Bocchi the Rock!,Animation,Japanese,Ended,25.0,2022-10-08,https://bocchi.rocks,41,https://api.tvmaze.com/shows/64321,...,0,0,0,0,0,0,0,0,0,0
1596,https://www.tvmaze.com/shows/1596/ufc-fight-night,UFC Fight Night,Sports,English,Running,190.0,2005-08-06,http://www.ufc.com/,92,https://api.tvmaze.com/shows/1596,...,0,0,0,0,0,0,0,0,0,0
1505,https://www.tvmaze.com/shows/1505/one-piece,One Piece,Animation,Japanese,Running,26.0,1999-10-20,http://www.toei-anim.co.jp/tv/onep/,99,https://api.tvmaze.com/shows/1505,...,0,0,0,0,0,0,0,0,0,0


In [17]:
# Comprobar si la carpeta data existe, si no, crearla
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

# Guardar los DataFrames en archivos Parquet con compresión snappy
episodes_df.to_parquet(os.path.join(data_dir, 'episodes.parquet'), compression='snappy')
episode_ratings_df.to_parquet(os.path.join(data_dir, 'episode_ratings.parquet'), compression='snappy')
shows_df.to_parquet(os.path.join(data_dir, 'shows.parquet'), compression='snappy')
show_schedule_df.to_parquet(os.path.join(data_dir, 'show_schedule.parquet'), compression='snappy')
show_ratings_df.to_parquet(os.path.join(data_dir, 'show_ratings.parquet'), compression='snappy')
webchannel_df.to_parquet(os.path.join(data_dir, 'webchannel.parquet'), compression='snappy')

# Leer los archivos Parquet
episodes_df = pd.read_parquet(os.path.join(data_dir, 'episodes.parquet'))
episode_ratings_df = pd.read_parquet(os.path.join(data_dir, 'episode_ratings.parquet'))
shows_df = pd.read_parquet(os.path.join(data_dir, 'shows.parquet'))
show_schedule_df = pd.read_parquet(os.path.join(data_dir, 'show_schedule.parquet'))
show_ratings_df = pd.read_parquet(os.path.join(data_dir, 'show_ratings.parquet'))
webchannel_df = pd.read_parquet(os.path.join(data_dir, 'webchannel.parquet'))

# Comprobar si la carpeta db existe, si no, crearla
if not os.path.exists(db_dir):
    os.makedirs(db_dir)

# Ruta de la base de datos SQLite
db_path = os.path.join(db_dir, 'data.db')


# Conexión a la base de datos SQLite
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

##Crear las tablas y definir las relaciones

# Crear la tabla episodes
cursor.execute("""
CREATE TABLE IF NOT EXISTS episodes (
    id INTEGER PRIMARY KEY,
    url TEXT,
    name TEXT,
    season INTEGER,
    number INTEGER,
    type TEXT,
    airstamp TEXT,
    runtime INTEGER,
    _links_self_href TEXT,
    _links_show_href TEXT
)""")
episodes_df.to_sql('episodes', conn, if_exists='append', index=False)

# Crear la tabla episode_ratings
cursor.execute("""
CREATE TABLE IF NOT EXISTS episode_ratings (
    id INTEGER PRIMARY KEY,
    rating_average REAL,
    FOREIGN KEY (id) REFERENCES episodes (id)
)""")
episode_ratings_df.to_sql('episode_ratings', conn, if_exists='append', index=False)

# Crear la tabla shows
cursor.execute("""
CREATE TABLE IF NOT EXISTS shows (
    _embedded_show_id INTEGER PRIMARY KEY,
    _embedded_show_url TEXT,
    _embedded_show_name TEXT,
    _embedded_show_type TEXT,
    _embedded_show_language TEXT,
    _embedded_show_status TEXT,
    _embedded_show_averageRuntime INTEGER,
    _embedded_show_premiered TEXT,
    _embedded_show_officialSite TEXT,
    _embedded_show_weight INTEGER,
    _embedded_show__links_self_href TEXT,
    Action INTEGER,
    Adult INTEGER,
    Adventure INTEGER,
    Anime INTEGER, 
    Children INTEGER,
    Comedy INTEGER,
    Crime INTEGER,
    DIY INTEGER,
    Drama INTEGER,
    Espionage INTEGER,
    Family INTEGER,
    Fantasy INTEGER,
    Food  INTEGER,
    History INTEGER,
    Horror  INTEGER,
    Legal INTEGER,
    Medical INTEGER,
    Music INTEGER,
    Mystery INTEGER,
    Nature  INTEGER,
    Romance INTEGER,
    Science_Fiction INTEGER,
    Sports INTEGER,
    Supernatural INTEGER,
    Thriller INTEGER,
    Travel INTEGER,
    War INTEGER,
    Western INTEGER
)""")
shows_df.to_sql('shows', conn, if_exists='append', index=False)

# Crear la tabla show_schedule
cursor.execute("""
CREATE TABLE IF NOT EXISTS show_schedule (
    _embedded_show_id INTEGER,
    _embedded_show_schedule_time TEXT,
    _embedded_show_schedule_days TEXT,
    FOREIGN KEY (_embedded_show_id) REFERENCES shows (_embeded_show_id)
)""")
show_schedule_df.to_sql('show_schedule', conn, if_exists='append', index=False)

# Crear la tabla show_ratings
cursor.execute("""
CREATE TABLE IF NOT EXISTS show_ratings (
    _embedded_show_id INTEGER,
    _embedded_show_rating_average REAL,
    FOREIGN KEY (_embedded_show_id) REFERENCES shows (_embeded_show_id)
)""")
show_ratings_df.to_sql('show_ratings', conn, if_exists='append', index=False)

# Crear la tabla webchannel
cursor.execute("""
CREATE TABLE IF NOT EXISTS webchannel (
    _embedded_show_id INTEGER,
    _embedded_show_webChannel_id INTEGER,
    _embedded_show_webChannel_name TEXT,
    _embedded_show_webChannel_country_name TEXT,
    FOREIGN KEY (_embedded_show_id) REFERENCES shows (_embeded_show_id)
)""")
webchannel_df.to_sql('webchannel', conn, if_exists='append', index=False)

# Confirmar los cambios y cerrar la conexión
conn.commit()
conn.close()

In [19]:
# a. Runtime promedio de los episodios
average_runtime = episodes_df['runtime'].mean()
print("Runtime promedio:", average_runtime, "minutos")

# b. Conteo de shows de TV por género
# Puedes obtener esto sumando las columnas correspondientes a cada género en el DataFrame de shows
genre_counts = shows_df[['Action', 'Adult', 'Adventure', 'Anime', 'Children', 'Comedy', 'Crime', 'DIY', 'Drama', 'Espionage', 'Family', 'Fantasy', 'Food', 'History', 'Horror', 'Legal', 'Medical', 'Music', 'Mystery', 'Nature', 'Romance', 'Science_Fiction', 'Sports', 'Supernatural', 'Thriller', 'Travel', 'War', 'Western']].sum()
print("Conteo de shows de TV por género:")
print(genre_counts)

# c. Listar los dominios (web) del sitio oficial de los shows
# Puedes obtener el dominio utilizando la librería urllib
from urllib.parse import urlparse

# Definir una función para extraer el dominio
def extract_domain(url):
    try:
        return urlparse(url).netloc
    except:
        return None

# Aplicar la función para obtener los dominios
domains = shows_df['_embedded_show_officialSite'].apply(extract_domain)

# Listar los dominios únicos
unique_domains = domains.dropna().unique()
print("Dominios (web) del sitio oficial de los shows:")
print(unique_domains)

Runtime promedio: 35.25736961451247 minutos
Conteo de shows de TV por género:
Action              37
Adult                5
Adventure           45
Anime               32
Children            28
Comedy             158
Crime               44
DIY                  7
Drama              174
Espionage            5
Family              18
Fantasy             57
Food                27
History             24
Horror               3
Legal                2
Medical              8
Music               13
Mystery             39
Nature               7
Romance             90
Science_Fiction     10
Sports              19
Supernatural         8
Thriller            34
Travel              12
War                  6
Western              1
dtype: int64
Dominios (web) del sitio oficial de los shows:
['hd.kinopoisk.ru' 'hoshiiro.jp' 'bocchi.rocks' 'www.ufc.com'
 'www.toei-anim.co.jp' 'v.qq.com' 'v.youku.com' 'www.youtube.com'
 'www.iqiyi.com' 'www.servusbaby-series.com' 'ent.abs-cbn.com'
 'independentwrestling.tv' 